In [1]:
import json
import random

In [2]:
percentage = '20'

In [3]:
with open('./createData/multiwoz21/data.json', 'r') as f:
    data = json.load(f)

In [4]:
with open('./data/multiwiz/user/'+percentage+'p/goals_new.json', 'r') as f:
    new_goal = json.load(f)
    
with open('./artificial_data/data_all_'+percentage+'p.json', 'r') as f:
    art_data = json.load(f)
    
with open('./artificial_data/query_all_'+percentage+'p.json', 'r') as f:
    art_query = json.load(f)

In [5]:
d_train= json.load(open('createData/multiwoz21/db/train_db.json'))
d_rest = json.load(open('createData/multiwoz21/db/restaurant_db.json'))
d_hotel = json.load(open('createData/multiwoz21/db/hotel_db.json'))
d_att = json.load(open('createData/multiwoz21/db/attraction_db.json'))
dom_db = {'train':d_train, 'restaurant':d_rest, 'hotel':d_hotel, 'attraction':d_att, 'taxi':d_train}

In [6]:
def get_goal(message):
#     message = goals['message']
    if isinstance(message, list):  
        message = ". ".join(message)
    message = " . ".join(message.split(". "))

    span_start = "class='emphasis'>"
    span_end = "</span>"
    span_start_len = len(span_start)
    span_end_len = len(span_end)
    
    words = message.split()
    msg = ""
    keys = []
    key_words = []
    i = 0
    j = 0
    while i < len(words):
        if words[i] == '<span':
            i += 1
            continue
        elif words[i].startswith(span_start):
            if span_end in words[i]:
                sind = len(msg) + 6
                end_idx = words[i].find(span_end)
                msg += '[key] ' + words[i][span_start_len:end_idx] + " [key] "
                keys.extend([sind, len(msg)-7])
                key_words.append(msg[sind:len(msg)-1])
                i += 1
            else:
                sind = len(msg) + 6
                msg += '[key] ' + words[i][span_start_len:] + " "
#                 msg.append(words[i][span_start_len:])
                keys.extend([sind, len(msg)-1])
                wk = [msg[sind:len(msg)-1]]
                i += 1
                while span_end not in words[i]:
                    sind = len(msg)
                    msg += words[i] + " "
                    keys.extend([sind, len(msg)-1])
                    wk.append(msg[sind:len(msg)-1])
                    i += 1
                sind = len(msg)  
                end_idx = words[i].find(span_end)
                msg += words[i][:end_idx] + " "
                keys.extend([sind, len(msg)-1])
                wk.append(msg[sind:len(msg)-1])
                key_words.append(" ".join(wk))
                msg += "[key] "
                i += 1
        else:
            sind = len(msg)
            msg += words[i] + " "
            i += 1   
    
    return msg, keys, key_words   

In [7]:
ng = []
for r in new_goal:
    t = r.lower()
    flag = True
    if t.find('hospital') != -1 or t.find('accident') != -1:
        flag = False
    elif t.find('police') != -1 or t.find('robbed') != -1:
        flag = False
    
    if flag:
        ng.append(r)

In [8]:
file_goal = {}
for d in data:
    msg, _, _ = get_goal(data[d]['goal']['message'])
    
    for i,g in enumerate(ng[:50]):
        if msg == g:
            file_goal[g] = {'file':d, 'idx':i}


In [9]:
def getBS(bs):
    dict_bs = {}
    bs = bs.replace('[Q]', '').split('|')[1:]
    for b in bs:
        b = b.split('=')
        if b[1].strip() != '*':
            dict_bs[b[0].strip()] = b[1].strip()
            
    return dict_bs

In [10]:
importantKeys = {}
importantKeys['train'] = dom_db['train'][0].keys()
importantKeys['hotel'] = dom_db['hotel'][0].keys()
importantKeys['restaurant'] = dom_db['restaurant'][0].keys()
importantKeys['attraction'] = dom_db['attraction'][0].keys()


def modify(art_data, art_bs):
#     d = d.replace('[taxi_', '[train_')
#     d = d.replace('[value_', '[restaurant_')

    modified_data = []
    bs_count = 0
    name_key = -1
    name_dom = ""
    for d in art_data:
        if '[User]'  in d:
            modified_data.append(d)
        else:
            d = d.replace('[taxi_', '[train_')
            bs = getBS(art_bs[bs_count])
            
            for dom in ['train', 'attraction', 'hotel', 'restaurant']:
                for key in importantKeys[dom]:
                    if '[value_'+key+']' in d:
                        if key in bs:
                            d = d.replace('[value_'+key+']', bs[key])
                    
            d = d.replace('[value_', '[restaurant_')       
            for dom in ['train', 'attraction', 'hotel', 'restaurant']:
                for key in importantKeys[dom]:
                    l = len(dom_db[dom])
                    if '['+dom+'_'+key+']' in d:
                        if key in bs:
                            d = d.replace('['+dom+'_'+key+']', bs[key])
                            
                    if name_key != -1:
                        if '['+dom+'_name]' in d:
                            d = d.replace('['+dom+'_'+key+']', dom_db[name_dom][name_key]['name'], 1)
                       
                    for i in range(10):
                        if '['+dom+'_'+key+']' in d:
                            rand = random.randint(0,l-1)
                            d = d.replace('['+dom+'_'+key+']', dom_db[dom][rand][key], 1)
                            if key == 'name':
                                name_key = rand
                                name_dom = dom

                    if '['+dom+'_count]' in d:
                        rand = random.randint(0,l-1)
                        d = d.replace('['+dom+'_count]', str(rand))

                    if '['+dom+'_place]' in d:
                        rand = random.randint(0,l-1)
                        d = d.replace('['+dom+'_place]', 'Cambridge')
            
            modified_data.append(d)
            bs_count += 1
    print(modified_data)
    return modified_data
            

In [11]:
final_data = {}

for g in file_goal:
    final_data[file_goal[g]['file']] = {}
    final_data[file_goal[g]['file']]['goal'] = g.replace('[key]', '').replace('  ', ' ')
    final_data[file_goal[g]['file']]['art_dial'] = modify(art_data[file_goal[g]['idx']][1:], art_query[file_goal[g]['idx']])
    final_data[file_goal[g]['file']]['orig_dial'] = []
    for l in data[file_goal[g]['file']]['log']:
        if bool(l['metadata']):
            print('$$', l['text'].strip().lower().replace("\n", ""))
            final_data[file_goal[g]['file']]['orig_dial'].append('[Agent] ' + l['text'].strip().lower().replace("\n", ""))
        else:
            final_data[file_goal[g]['file']]['orig_dial'].append('[User] ' + l['text'].strip().replace("\n", ""))
            

['[User] i need a train to cambridge on saturday.', '[Agent] where are you departing from ?', "[User] i'll be departing from norwich.", '[Agent] there are 16 trains leaving Cambridge on saturday , is there a specific time you would like to leave ?', '[User] i would like to arrive by 19:00.', '[Agent] train [train_id] arrives at [restaurant_time] . would you like me to book this train for you ?', '[User] yes please. i need the travel time.', '[Agent] the travel time is 29 minutes .', '[User] great. i also need a place to stay.', '[Agent] i have 83 hotel -s in the east . do you have a price range ?', '[User] no, i just need it to be 4 stars and not parking.', '[Agent] okay , how about the the cambridge belfry ?', '[User] that sounds great. can you book it for 5 people for 4 nights starting saturday?', '[Agent] i have booked that for you . your reference number is [hotel_reference] .', '[User] thank you so much. that is all i needed.', '[Agent] you are welcome , enjoy your stay in Cambrid

$$ i have several trains available.  what time would you like to depart?
$$ reference number is : mo8sec17.  train id tr8674. is there anything else?
$$ unfortunately there is no malaysian food in the centre. do you want a different type of food, or would you like me to search a different part of town?
$$ sure, there are a number of options.  did you have a certain price range in mind?
$$ i would suggest curry garden. for what day should i make your reservation?
$$ your reservation for 6 people at curry garden this saturday at 12:45 was successful. the table will be reserved for 15 minutes. your reference number is : aen16uhv.
$$ thank you for contacting us and have a nice day.
["[User] i am looking for a place to stay. the hotel doesn't need to have free parking and doesn't need to include internet.", '[Agent] i do not have any expensive hotel -s in town . is there another price range you would like to stay in ?', '[User] no, i just need the address.', '[Agent] the address is warkwort

In [12]:
final_data

{'MUL2029.json': {'goal': "You are looking for information in Cambridge . You are looking for a train . The train should arrive by 19:00 and should leave on saturday . The train should depart from norwich and should go to cambridge . Make sure you get travel time . You are also looking for a place to stay . The hotel should have a star of 4 and doesn't need to have free parking . The hotel should be in the moderate price range . Once you find the hotel you want to book it for 5 people and 4 nights starting from the same day . Make sure you get the reference number ",
  'art_dial': ['[User] i need a train to cambridge on saturday.',
   '[Agent] where are you departing from ?',
   "[User] i'll be departing from norwich.",
   '[Agent] there are 16 trains leaving Cambridge on saturday , is there a specific time you would like to leave ?',
   '[User] i would like to arrive by 19:00.',
   '[Agent] train [train_id] arrives at [restaurant_time] . would you like me to book this train for you ?'

In [13]:
with open('evaluation_data.json', 'w') as f:
    json.dump(final_data, f, indent=4)

In [14]:
ng[50]

'You are planning your trip in Cambridge . You are looking for a [key] train [key] . The train should go to [key] london liverpool street [key] and should depart from [key] cambridge [key] . The train should leave on [key] friday [key] and should [key] leave after 10:30 [key] . Make sure you get [key] price [key] . You are also looking for a [key] place to stay [key] . The hotel should [key] include free wifi [key] and should be in the [key] expensive [key] price range . The hotel should be in the type of [key] guesthouse [key] . If there is no such hotel, how about one that is in the [key] moderate [key] price range . Make sure you get [key] postcode [key] and [key] address [key] '

In [15]:
art_data[50]

['[st@rt]',
 '[User] i need a train to london liverpool street.',
 '[Agent] where will you be departing from and what day will you be traveling ?',
 '[User] i will be departing from cambridge on friday.',
 '[Agent] i have train [train_id] that leaves at [value_time] and arrives at [value_time] . would you like me to book this for you ?',
 '[User] yes please. i need the price, please.',
 '[Agent] the price is [value_price] pounds',
 '[User] great. i also need a place to stay.',
 '[Agent] i have [value_count] guesthouses and [value_count] hotel , which are in the [value_pricerange] price range ?',
 '[User] i would prefer a guesthouse.',
 '[Agent] i am sorry , there are no [value_pricerange] guesthouses in town . would you like to try a different price range or area ?',
 '[User] how about a moderately priced guesthouse?',
 '[Agent]  [hotel_name] is a [value_pricerange] -ly priced guesthouse in the [value_area] . would you like me to make a reservation ?',
 '[User] yes please. i need the p

In [16]:
file_goal

{"You are looking for information in Cambridge . You are looking for a [key] train [key] . The train should [key] arrive by 19:00 [key] and should leave on [key] saturday [key] . The train should depart from [key] norwich [key] and should go to [key] cambridge [key] . Make sure you get [key] travel time [key] . You are also looking for a [key] place to stay [key] . The hotel should have [key] a star of 4 [key] and [key] doesn't need to have free parking [key] . The hotel should be in the [key] moderate [key] price range . Once you find the [key] hotel [key] you want to book it for [key] 5 people [key] and [key] 4 nights [key] starting from [key] the same day [key] . Make sure you get the [key] reference number [key] ": {'file': 'MUL2029.json',
  'idx': 0},
 'You are looking for information in Cambridge . You are looking for a [key] place to dine [key] . The restaurant should serve [key] european [key] food and should be in the [key] expensive [key] price range . Once you find the [key]

In [17]:
art_data

[['[st@rt]',
  '[User] i need a train to cambridge on saturday.',
  '[Agent] where are you departing from ?',
  "[User] i'll be departing from norwich.",
  '[Agent] there are [value_count] trains leaving [value_place] on [value_day] , is there a specific time you would like to leave ?',
  '[User] i would like to arrive by 19:00.',
  '[Agent] train [train_id] arrives at [value_time] . would you like me to book this train for you ?',
  '[User] yes please. i need the travel time.',
  '[Agent] the travel time is [value_count] minutes .',
  '[User] great. i also need a place to stay.',
  '[Agent] i have [value_count] hotel -s in the [value_area] . do you have a price range ?',
  '[User] no, i just need it to be 4 stars and not parking.',
  '[Agent] okay , how about the [hotel_name] ?',
  '[User] that sounds great. can you book it for 5 people for 4 nights starting saturday?',
  '[Agent] i have booked that for you . your reference number is [hotel_reference] .',
  '[User] thank you so much. 

In [18]:
data[list(data.keys())[0]]

{'goal': {'taxi': {},
  'police': {},
  'hospital': {},
  'hotel': {'info': {'type': 'hotel',
    'parking': 'yes',
    'pricerange': 'cheap',
    'internet': 'yes'},
   'fail_info': {},
   'book': {'pre_invalid': True,
    'stay': '2',
    'day': 'tuesday',
    'invalid': False,
    'people': '6'},
   'fail_book': {'stay': '3'}},
  'topic': {'taxi': False,
   'police': False,
   'restaurant': False,
   'hospital': False,
   'hotel': False,
   'general': False,
   'attraction': False,
   'train': False,
   'booking': False},
  'attraction': {},
  'train': {},
  'message': ["You are looking for a <span class='emphasis'>place to stay</span>. The hotel should be in the <span class='emphasis'>cheap</span> price range and should be in the type of <span class='emphasis'>hotel</span>",
   "The hotel should <span class='emphasis'>include free parking</span> and should <span class='emphasis'>include free wifi</span>",
   "Once you find the <span class='emphasis'>hotel</span> you want to book it